# SAT Bootcamp Analysis 2
### Goal:
Evenly divide new Ckodon students into random groups of 30.
Assign group labels from I to Z.

### Steps
1. Import Google Form data into Pandas DataFrame.
2. Clean the data:
    1. Clean leading and trailing white space from all string columns.
    2. Search for and print out duplicate rows.
    3. Search for and print out invalid email addresses.
3. Group the students:
    1. Define function for randomly assigning groups.
    2. Create \`SAT Group\` column and assign groups in column.
    3. Sort students by group.
4. Export resulting DataFrame to Excel Workbook.

Future
1. Replace group naming with dictionary comprehension
2. Remove duplicates after white space extraction
3. Move code for sending email to different module.
4. Remove hardcoded WhatsApp links plus app password.

## Import Google Form data

In [3]:
# import required modules
import pandas as pd
from random import randint

# import students
students_bf = pd.read_excel("../data/Ckodon Activity Review Form - Before (Responses).xlsx")
students_ng = pd.read_excel("../data/Ckodon Activity Review Form - No Group (Responses).xlsx")


# test import success
print(students_bf.head())
print(students_ng.head())

                Timestamp            Full Name Gender WhatsApp Number  \
0 2023-07-10 19:12:48.333      Amoako Raymond    Male      0545758924   
1 2023-07-10 19:14:55.963     Francis Brobbey    Male       550144052   
2 2023-07-10 19:15:25.498   Ransford Kofi Teng   Male      0555568395   
3 2023-07-10 19:16:53.994    Philip Tenkorang    Male      0246621075   
4 2023-07-10 19:16:57.521  Prince Osei Boakye    Male   +233554230594   

                          Email  
0         raymonda883@gmail.com  
1   francisbrobbey23@gmail.com   
2      tengransford65@gmail.com  
3  philiptenkorang123@gmail.com  
4  oseiboakyeprince39@gmail.com  
                Timestamp                      Full Name  Gender  \
0 2023-07-12 22:02:06.623          Owusu Rexford Yeboah     Male   
1 2023-07-12 22:03:56.614                 Bernard Owusu     Male   
2 2023-07-12 22:04:17.678  Cindy Nana Yaa Adomah Oppong   Female   
3 2023-07-12 22:04:45.383          Derrick Selasi Lawer     Male   
4 2023-07-12 22:0

## Clean the data

### Remove leading and trailing white space from all string columns.
Remove leading and trailing whitespace from the `Full Name` and `Email` column.


In [4]:
students_bf["Full Name"] = students_bf["Full Name"].apply(str.strip)
students_bf["Email"] = students_bf["Email"].apply(str.strip)
students_ng["Full Name"] = students_ng["Full Name"].apply(str.strip)
students_ng["Email"] = students_ng["Email"].apply(str.strip)

### Search and print out duplicate rows
Identify all rows with the same Full Name and Email or Full Name and WhatsApp Number.
Duplicate rows will be manually inspected in MS Excel and removed from the data set after identification.


In [5]:
# define function for finding duplicates
def print_duplicates(dataframe):
    """Prints all duplicate rows in `dataframe`.

    Parameters
    ==========
    dataframe_ : pandas.core.frame.DataFrame
        A pandas DataFrame containing each student's information.

    Returns
    =======
    None
    """
    # create copy of `dataframe`
    dataframe_cp = dataframe.copy()

    # extract relevant columns as lists for easy searching
    dataframe_cp["Full Name"] = dataframe_cp["Full Name"].apply(str.lower)  # remove casing
    dataframe_cp["Email"] = dataframe_cp["Full Name"].apply(str.lower)  # remove casing
    names = [name.lower() for name in dataframe_cp["Full Name"]]
    emails = [email.lower() for email in dataframe_cp["Email"]]
    whatsapp_nos = list(dataframe_cp["WhatsApp Number"])

    for row in dataframe_cp.index:
        name = dataframe_cp.loc[row]["Full Name"]
        email = dataframe_cp.loc[row]["Email"]
        whatsapp_no = dataframe_cp.loc[row]["WhatsApp Number"]
        row_excel = row + 2  # row number in MS Excel

        if names.count(name) > 1:
            if emails.count(email) > 1 or whatsapp_nos.count(whatsapp_no) > 1:
                no_of_duplicates = names.count(name)
                print(
                    f"DUPLICATE FOUND ({no_of_duplicates}): {row_excel} {dataframe['Full Name'][row]} {whatsapp_no} {dataframe['Email'][row]}")

In [6]:
# search duplicate values from `students_bf`
print("----------Searching-STUDENTS_BF--------------")
print_duplicates(students_bf)

# search duplicate values from `students_ng`
print("\n\n----------Searching-STUDENTS_NG--------------")
print_duplicates(students_ng)

----------Searching-STUDENTS_BF--------------


----------Searching-STUDENTS_NG--------------


In [7]:
# merge and search duplicates across sheets (after removing duplicates manually)
students = pd.concat([students_bf, students_ng], ignore_index=True)

print("----------Searching-STUDENTS_BF-and-STUDENTS_NG--------------")
print_duplicates(students)

----------Searching-STUDENTS_BF-and-STUDENTS_NG--------------


### Search and print out invalid email addresses
Crucial as each student's group will be sent to them via email.

In [8]:
# define function for validating email addresses
from re import search


def validate_address(email):
    """Return True if `email` is valid or False otherwise.

    Parameters
    ==========
    email: str
        The email address to be validated.

    Returns
    =======
    bool
        True means email address is valid. False means email address is invalid.
    """

    pattern = "[a-zA-Z0-9\-.+_]+@[a-zA-Z0-9\-.+_]+\.[a-zA-Z]{2,}"
    match = search(pattern, email)
    return bool(match)

In [9]:
# call function on all addresses in email column

# students_bf
print("-------Invalid addresses in `students_bf`--------")
for row in students_bf.index:
    email = students_bf.loc[row]["Email"]
    if not validate_address(email):
        print(students_bf.loc[row])

# students_ng
print("-------Invalid addresses in `students_ng`--------")
for row in students_ng.index:
    email = students_ng.loc[row]["Email"]
    if not validate_address(email):
        print(students_ng.loc[row])

-------Invalid addresses in `students_bf`--------
-------Invalid addresses in `students_ng`--------


## Group the students

### Define function for randomly assigning groups

In [10]:
# recursive random number generator without repetition
previous = []


def generate(a, b):
    """Returns pseudorandom integer between a and b if not already contained in a global list with name `previous`.

    Parameters
    ==========
    a : int
        Lower bound of the range within which generated number may lie (inclusive).
    b : int
        Upper bound of the range within which generated number may lie (inclusive).

    Returns
    =======
    int
        A pseudorandom number between a and b inclusive whose value is not already contained in `previous` list.
    """
    number = randint(a, b)
    if number not in previous:
        previous.append(number)
        return number
    else:
        return generate(a, b)

### Create \`SAT Group\` column and assign groups in column

In [11]:
# Create SAT Group column
students_bf["SAT Group"] = None
students_ng["SAT Group"] = None

In [12]:
# Assign groups for `students_bf`
no_of_groups_a = len(students_bf) // 30  # no of groups for students_bf
groups = [chr(i + 73) for i in range(no_of_groups_a)]

# for the no. of students there are, pick a random student and assign them a group.
for i in range(len(students_bf)):
    student = generate(0, len(students_bf) - 1)
    group = groups[i % len(groups)]
    students_bf["SAT Group"][student] = group

print("---------No-of-Students-per-Group--------")
for group in groups:
    print(group, list(students_bf["SAT Group"]).count(group))

---------No-of-Students-per-Group--------
I 30
J 30
K 30
L 30
M 30


C:\Users\Francis O'Hara Aidoo\AppData\Local\Temp\ipykernel_24596\1751309296.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  students_bf["SAT Group"][student] = group
C:\Users\Francis O'Hara Aidoo\AppData\Local\Temp\ipykernel_24596\1751309296.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  students_bf["SAT Group"][student] = group
C:\Users\Francis O'Hara Aidoo\AppData\Local\Temp\ipykernel_24596\1751309296.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

In [13]:
# Assign groups for `students_ng`
previous = []  # reset `previous` list for random number generator
students_ng["SAT Group"] = None  # create new column
no_of_groups_b = len(students_ng) // 30  # no of groups for students_bf
groups = [chr(i + 73 + no_of_groups_a) for i in range(no_of_groups_b)]

# for the no. of students there are, pick a random student and assign them a group.
for i in range(len(students_ng)):
    student = generate(0, len(students_ng) - 1)
    group = groups[i % len(groups)]
    students_ng["SAT Group"][student] = group

print("---------No-of-Students-per-Group--------")
for group in groups:
    print(group, list(students_ng["SAT Group"]).count(group))

C:\Users\Francis O'Hara Aidoo\AppData\Local\Temp\ipykernel_24596\2846154827.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  students_ng["SAT Group"][student] = group
C:\Users\Francis O'Hara Aidoo\AppData\Local\Temp\ipykernel_24596\2846154827.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  students_ng["SAT Group"][student] = group
C:\Users\Francis O'Hara Aidoo\AppData\Local\Temp\ipykernel_24596\2846154827.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

---------No-of-Students-per-Group--------
N 32
O 32
P 32
Q 32
R 32
S 31


### Sort students by group

In [14]:
students_bf.sort_values("SAT Group", inplace=True)
students_ng.sort_values("SAT Group", inplace=True)

print(students_bf)
print(students_ng)

                  Timestamp                     Full Name  Gender  \
0   2023-07-10 19:12:48.333                Amoako Raymond    Male   
37  2023-07-11 01:15:30.645                    Holy Agyei    Male   
38  2023-07-11 01:34:35.804        Maureen Sena Ahiamatah  Female   
39  2023-07-11 02:16:52.008                 Maxwell Offin    Male   
43  2023-07-11 07:56:05.284                   Asum Victor    Male   
..                      ...                           ...     ...   
105 2023-07-17 05:06:45.690               Vayant Offeibea  Female   
106 2023-07-17 05:26:58.991                 Asare Michael    Male   
110 2023-07-17 08:07:06.983  Michael Obinna Emmanuel Kalu    Male   
120 2023-07-17 21:56:03.394         Alhassan Nuratu Lagfu  Female   
74  2023-07-13 07:39:46.581            Aborsi Philip Teye    Male   

    WhatsApp Number                           Email SAT Group  
0        0545758924           raymonda883@gmail.com         I  
37       0555161098            agyeiholy5@g

## Export to Excel Workbook

In [15]:
students_bf.to_excel("../data/output/Ckodon Activity Review Groups(BEFORE).xlsx", index=False)
students_ng.to_excel("../data/output/Ckodon Activity Review Groups(NO GROUP).xlsx", index=False)

# Email Group Assignments to Students
Email the assigned group and corresponding WhatsApp group link to each student.

In [19]:
# concatenate `students_bf` and `students_ng`
students = pd.concat([students_bf, students_ng], ignore_index=True)
students

,Timestamp,Full Name,Gender,WhatsApp Number,Email,SAT Group
0,2023-07-10 19:12:48.333,Amoako Raymond,Male,0545758924,raymonda883@gmail.com,I
1,2023-07-11 01:15:30.645,Holy Agyei,Male,0555161098,agyeiholy5@gmail.com,I
2,2023-07-11 01:34:35.804,Maureen Sena Ahiamatah,Female,0540713430,Ahiamatahsenamaureen@gmail.com,I
3,2023-07-11 02:16:52.008,Maxwell Offin,Male,+233257154710,maxwelloffin267@gmail.com,I
4,2023-07-11 07:56:05.284,Asum Victor,Male,0547113025,Iamasum369@gmail.com,I
...,...,...,...,...,...,...
336,2023-07-16 13:20:32.876,Ibrahim Suhiyini Abdul-Aziz,Male,0550984454,syhib18@gmail.com,S
337,2023-07-12 22:26:25.284,Agyei Clement,Male,0593596889,www. agyeiclement585@gmail.com,S
338,2023-07-16 22:41:17.691,Nkoah Joseph,Male,0599249794,joshgrey345@gmail.com,S
339,2023-07-14 16:38:56.610,Nancy Simpson,Female,0201319179,nancysimpson546@gmail.com,S


In [ ]:
# import required modules
from smtplib import SMTP
from email.message import EmailMessage
from time import sleep


# set WhatsApp links for each group
whatsapp_link = {
                    "I": "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
                    "J": "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
                    "K": "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
                    "L": "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
                    "M": "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
                    "N": "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
                    "O": "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
                    "P": "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
                    "Q": "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
                    "R": "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
                    "S": "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
                    }

# instantiate SMTP client
server = "smtp.gmail.com"
port = "587"
username = "ckodontech@gmail.com"
password = input("Enter the App password you received from Ckodon:")

smtp = SMTP(server, port)
smtp.ehlo()
smtp.starttls()
smtp.login(username, password)


# to prevent server overload (Gmail SMTP Error 421)
emails_sent = 0  # no. of emails sent so far
delay_s = 60  # delay in seconds


# for every student, create and send an email message
for row in students.index:
    student = students.loc[row]
    msg = EmailMessage()
    msg["From"] = "ckodontech@gmail.com"
    msg["To"] = "franciskohara@gmail.com"# student_bf["Email"]
    msg["Subject"] = "Your Ckodon Activity Review Group"
    body = f"""Dear {student["Full Name"].title()},

You have been assigned to Ckodon Activity Review Group {student["SAT Group"]}.

You may use the link below to join your assigned group on WhatsApp:
{whatsapp_link[student["SAT Group"]]}

Join the group as soon as possible so that we can begin with the review. This link is meant to be used by you alone. Do not share with any other person.

Please do not reply to this email.

Best,
The Ckodon Foundation Team."""
    msg.set_content(body)

    # send email message
    try:
        smtp.send_message(msg)
        print("SENT")
        emails_sent += 1

        # wait 30s every 60 emails to prevent Gmail SMTP Error 421
        if emails_sent == 60:
            emails_sent = 0
            sleep(delay_s)
    except Exception as exception:
        print("Exception:", exception)
        print("Sending terminated at row number:", row)
